In [ ]:
cov = 1  # 1 means simple (0.03x), 2 means double (0.06x), etc.
slice='B'

# base_dir = f'/Users/dd/work/svc/data/silver_{slice}_5_05'
base_dir = f'/Users/dd/work/svc/data/gastric'
clusters = open(f'{base_dir}/spectral_clustering').readlines()[0][:-1].split(',')
gt_lines = open(f'{base_dir}/../mapping{slice}.tsv').readlines()

# contains <cell_id, ground_truth, cluster>, where "ground truth" is the result of CNV analysis
result = open(f'{base_dir}/../cell_id_gt_spectral', 'w')

# barcode_to_idx_lines = open(f'{base_dir}/../barcode_to_idx_{slice}.map').readlines()
barcode_to_idx_lines = open(f'{base_dir}/barcode_to_idx.map').readlines()

print(
    f'{len(clusters)} clusters, {len(barcode_to_idx_lines)} barcode->id mappings, {len(gt_lines)} ground truth entries')

barcode_to_cell_idx = {}
cell_idx_to_barcode = {}
for line in barcode_to_idx_lines:
    barcode, cell_idx = line.split('\t')
    barcode = barcode.split('_')[-1]
    barcode = barcode.split('-')[0]
    cell_idx = int(cell_idx[:-1])
    barcode_to_cell_idx[barcode] = cell_idx
    cell_idx_to_barcode[cell_idx] = barcode

barcode_to_cell_group = {}
for gt_line in gt_lines:
    gt_line = gt_line.split('\t')
    cell_id_str = gt_line[0]
    cell_group = gt_line[2][:-1]
    barcode_to_cell_group[cell_id_str] = cell_group

group_count = 3

id_to_cluster_a = {}
id_to_cluster_b = {}
id_to_cluster_c = {}

other_clusters=0
for i in range(0, len(clusters)):
    cell_id_str = cell_idx_to_barcode[i]
    if cell_id_str not in barcode_to_cell_group:
        print(cell_id_str)
    cell_group = '1' # barcode_to_cell_group[cell_id_str]

    result.write(cell_id_str + '\t' + cell_group + '\t' + clusters[i] + '\n')

    if int(clusters[i]) == 0:
        if cell_group not in id_to_cluster_a:
            id_to_cluster_a[cell_group] = 1
        else:
            id_to_cluster_a[cell_group] += 1
    elif int(clusters[i]) == 1:
        if cell_group not in id_to_cluster_b:
            id_to_cluster_b[cell_group] = 1
        else:
            id_to_cluster_b[cell_group] += 1
    elif int(clusters[i]) == 2:
        if cell_group not in id_to_cluster_c:
            id_to_cluster_c[cell_group] = 1
        else:
            id_to_cluster_c[cell_group] += 1
    else:
        other_clusters+=1

print("Other clusters: ", other_clusters)

for id, count_a in id_to_cluster_a.items():
    count_b = 0
    count_c = 0
    if id in id_to_cluster_b:
        count_b = id_to_cluster_b[id]
        del id_to_cluster_b[id]
    if id in id_to_cluster_c:
        count_c = id_to_cluster_c[id]
        del id_to_cluster_c[id]
    if count_c > 0:
        print(f'{id}: {count_a}/{count_b}/{count_c}')
    else:
        print(f'{id}: {count_a}/{count_b}')

print('==============')

for id, count_b in id_to_cluster_b.items():
    print(f'{id}: 0/{count_b}')

print('==============')

for id, count_c in id_to_cluster_c.items():
    print(f'{id}: 0/{count_c}')


In [ ]:
# plotting clusters for the gastric cell line; similar to the 2nd section in this notebook
import matplotlib.pyplot as plt
import numpy as np


base_dir = f'/Users/dd/work/svc/data/gastric/'

eigenvectors = np.loadtxt(f'{base_dir}/sim_mat_eigenvectors_norm.csv')

#id_to_group = open(f'{base_dir}/../breast_group_{cov}').readlines()[0].split(',')

# different from clustering_info above: it only contains the cells that were actually processed, together
# with the gt clustering *and* our clustering
clustering_info = open(f'{base_dir}/../cell_id_gt_spectral').readlines()

print(f'Gt lines {len(clustering_info)}')

# for each type of cell (e.g. Clone159, Clone19), contains the (x,y) coordinates given by the 2nd and 3rd eigenvectors
# this allows us to visualize if the clusters are separable in 2D
cell_name_to_xy = {}

x = eigenvectors[:, 1]
y = eigenvectors[:, 2]
num_cells = x.shape[0]

x1 = []
y1 = []
x2 = []
y2 = []


print('Eigenvectors size: ', x.shape[0], '. Cells to process: ', len(clustering_info))

j = 0
for i in range(0, len(clustering_info)):
    gt_str = clustering_info[i].split('\t')
    cell_barcode = gt_str[0]
    cluster_name = gt_str[2]
    cluster = int(gt_str[2])

    if cluster == 16383:  # cell doesn't belong to current subcluster
        continue

    idx = i # int(id_to_group[i])

    if idx >= num_cells:  # shouldn't happen?
        j += 1
        continue

    
    if cluster_name not in cell_name_to_xy:
        cell_name_to_xy[cluster_name] = ([x[idx]], [y[idx]])
    else:
        cell_name_to_xy[cluster_name][0].append(x[idx])
        cell_name_to_xy[cluster_name][1].append(y[idx])
       

    if cluster == 0:
        x1.append(x[idx])
        y1.append(y[idx])
    else:
        x2.append(x[idx])
        y2.append(y[idx])

print('Skipped ', j, ' records.')

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].scatter(x1, y1, label='tumor', color='red')
axs[0].scatter(x2, y2, label='healthy', color='blue')
axs[0].set(xlabel='2nd eigenvector', ylabel='3rd eigenvector')

# fig, axs = plt.subplots(1)

axs[1].scatter(x2, y2, label='healthy', color='blue')
axs[1].scatter(x1, y1, label='tumor', color='red')
axs[1].set(xlabel='2nd eigenvector', ylabel='3rd eigenvector')

fig, axs = plt.subplots(1)

for k, (v1, v2) in cell_name_to_xy.items():
    if k=='None':
        axs.scatter(v1, v2, label=k, alpha=0.1)
    else:
        axs.scatter(v1, v2, label=k)

# This plots the "None" cells at the end
# (v1,v2) = cell_name_to_xy['None']
# axs.scatter(v1, v2, label='None')

axs.set(xlabel='2nd eigenvector', ylabel='3rd eigenvector')
plt.gca().axes.get_xaxis().set_visible(False)
plt.gca().axes.get_yaxis().set_visible(False)
axs.legend()
